In [1]:
# make csv ready to datasets fromat
import pandas as pd

d_a = pd.read_csv("data/twitter_data/labels_tweets_dropna.csv")

from sklearn.model_selection import train_test_split
d_a['label']=(d_a.iloc[:,1:8].sum(axis=1) > 0).astype(int)

d_train, d_test = train_test_split(d_a_r, test_size=0.8, random_state=42)
d_test1, d_val = train_test_split(d_test, test_size=0.5, random_state=42)
d_train.shape, d_test1.shape, d_val.shape
d_train.iloc[:,1:].to_csv("data/twitter_data/tweet_train.csv",index=False)
d_test1.iloc[:,1:].to_csv("data/twitter_data/tweet_test.csv",index=False)
d_val.iloc[:,1:].to_csv("data/twitter_data/tweet_val.csv",index=False)

NameError: name 'd_a_r' is not defined

In [2]:
from datasets import load_dataset

data_files = {"train": "data/twitter_data/tweet_train.csv", "test": "data/twitter_data/tweet_test.csv", "val": "data/twitter_data/tweet_val.csv"}
dataset = load_dataset("csv", data_files=data_files)


Using custom data configuration default-15a9cf023f4f5ab3
Found cached dataset csv (C:/Users/bruger/.cache/huggingface/datasets/csv/default-15a9cf023f4f5ab3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["Tweet"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at C:/Users/bruger/.cache/huggingface/datasets/csv/default-15a9cf023f4f5ab3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-76d2be34d065f8a0.arrow
Loading cached processed dataset at C:/Users/bruger/.cache/huggingface/datasets/csv/default-15a9cf023f4f5ab3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-f0bf813d0579937c.arrow


  0%|          | 0/10 [00:00<?, ?ba/s]

In [6]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

Loading cached shuffled indices for dataset at C:/Users/bruger/.cache/huggingface/datasets/csv/default-15a9cf023f4f5ab3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-d0c4c758af366569.arrow
Loading cached shuffled indices for dataset at C:/Users/bruger/.cache/huggingface/datasets/csv/default-15a9cf023f4f5ab3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-c2d45e0058f3a5cb.arrow


In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [9]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [10]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [11]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [13]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Tweet. If Tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\bruger\miniconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 100
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 39
  Number of trainable parameters = 108311810


  0%|          | 0/39 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Tweet. If Tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.4103652834892273, 'eval_accuracy': 0.87, 'eval_runtime': 160.0922, 'eval_samples_per_second': 0.625, 'eval_steps_per_second': 0.081, 'epoch': 1.0}
